### Installing Deep SORT Real-time Library

In [ ]:
pip install deep_sort_realtime

Note: you may need to restart the kernel to use updated packages.


### Installing Ultralytics Library

In [ ]:
!pip install ultralytics

#Setting Environment Variable to Allow Duplicate KMP Libraries


In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

#Python Code for Real-time Object Tracking with YOLO and Deep SORT


In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

CONFIDENCE_THRESHOLD = 0.9
CAR_CLASS_ID = 2  # Assuming car class ID is 0
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)

video_cap = cv2.VideoCapture("/kaggle/input/aakash123/VIDEO-2024-03-16-23-08-40.mp4")
ret, frame = video_cap.read()
writer = cv2.VideoWriter("out.mp4", cv2.VideoWriter_fourcc(*'MP4V'), video_cap.get(cv2.CAP_PROP_FPS),
                          (frame.shape[1], frame.shape[0]))

model = YOLO("yolov8n.pt")
tracker = DeepSort(max_age=20)

car_count = 0  # Initialize car count
prev_car_count = 0  # Initialize previous car count
max_id=0
while True:
    ret, frame = video_cap.read()

    if not ret:
        break

    detections = model(frame)[0]
    results = []

    for data in detections.boxes.data.tolist():
        confidence = data[4]

        if float(confidence) < CONFIDENCE_THRESHOLD:
            continue

        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        class_id = int(data[5])

        if class_id == CAR_CLASS_ID:
            results.append([[xmin, ymin, xmax - xmin, ymax - ymin], confidence, class_id])

    tracks = tracker.update_tracks(results, frame=frame)
    car_count = len(tracks)  # Update car count based on the number of tracks

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        max_id=max(max_id,int(track_id))

        xmin, ymin, xmax, ymax = int(ltrb[0]), int(
            ltrb[1]), int(ltrb[2]), int(ltrb[3])

        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), GREEN, 2)
        cv2.rectangle(frame, (xmin, ymin - 20), (xmin + 20, ymin), GREEN, -1)
        cv2.putText(frame, str(track_id), (xmin + 5, ymin - 8),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 2)

    # Display car count on the frame
    cv2.putText(frame, f"Total cars detected: {max_id}", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8)

    writer.write(frame)

    prev_car_count = car_count  # Update previous car count

video_cap.release()
writer.release()


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'



0: 384x640 6 persons, 2 cars, 4 motorcycles, 1 truck, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 3 cars, 3 motorcycles, 1 truck, 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 4 cars, 4 motorcycles, 1 truck, 6.5ms
Speed: 2.2ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 5 cars, 2 motorcycles, 1 truck, 6.5ms
Speed: 1.7ms preprocess, 6.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 5 cars, 4 motorcycles, 1 truck, 6.4ms
Speed: 1.7ms preprocess, 6.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 4 cars, 4 motorcycles, 1 truck, 6.6ms
Speed: 1.9ms preprocess, 6.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 4 cars, 3 motorcycles, 1 truck, 6